# API Face Samples

## Objective

This script manages customer checkin data within a system by utilizing Azure Dynamic Person Group. It automates the detection and enrollment of customer faces, identifies customers within dynamic person groups (such as daily and location-specific groups), and links customers to these groups to streamline face recognition processes.

In [ ]:
import os
import logging
from shared_functions import (
    add_person_face, 
    create_person,
    create_dynamic_person_group,
    check_dynamic_person_group_exists,
    detect_faces,
    identify_faces,
    link_person_to_dynamic_person_group
)
from datetime import datetime

# Replace with your Azure Face API subscription key and endpoint
FACE_KEY = os.environ["FACE_API_KEY"]
FACE_ENDPOINT = os.environ["FACE_ENDPOINT_URL"]

image_path = "path_to_image.jpg"
patient_name = "Patient Name" # Optional
customer_group_id = "customer_group_id "
daily_group_id = f"daily_group_{customer_group_id}_{datetime.now().strftime('%Y_%m_%d')}"

logging.basicConfig(level=logging.INFO)

# Ensure both daily and customer dynamic groups exist
if not check_dynamic_person_group_exists(FACE_KEY, FACE_ENDPOINT, daily_group_id, "sample=customer_checkin_management"):
    create_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, daily_group_id,  "sample=customer_checkin_management")
    logging.info(f"Created dynamic group: {daily_group_id}")

if not check_dynamic_person_group_exists(FACE_KEY, FACE_ENDPOINT, customer_group_id, "sample=customer_checkin_management"):
    create_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, customer_group_id, "sample=customer_checkin_management")
    logging.info(f"Created customer group: {customer_group_id}")

# Detect faces in the image
detected_faces = detect_faces(FACE_KEY, FACE_ENDPOINT, image_path, "sample=customer_checkin_management")
if len(detected_faces) == 0:
    print("No faces detected in the image")
    exit()

# Check if the face exists in the daily group
face_id = detected_faces[0]["faceId"]
person_id = identify_faces(FACE_KEY, FACE_ENDPOINT, face_id, daily_group_id, "sample=customer_checkin_management")

if not person_id:
    logging.info("Person not found in daily group.")
    # Check if the face exists in the customer group
    person_id = identify_faces(FACE_KEY, FACE_ENDPOINT, face_id, customer_group_id, "sample=customer_checkin_management")

    if person_id:
        logging.info("Person exists in customer group.")
        link_person_to_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, daily_group_id, person_id, "sample=customer_checkin_management")
        logging.info("Person linked to daily group.")
    else:
        # If person not found, create a new person and add their face
        logging.info("Person does not exist in daily or customer group. Creating new person.")
        person_id = create_person(FACE_KEY, FACE_ENDPOINT, patient_name, "sample=customer_checkin_management")
        add_person_face(FACE_KEY, FACE_ENDPOINT, image_path, person_id, "sample=customer_checkin_management")

        # Link the person to the daily and customer group
        link_person_to_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, daily_group_id, person_id, "sample=customer_checkin_management")
        link_person_to_dynamic_person_group(FACE_KEY, FACE_ENDPOINT, customer_group_id, person_id, "sample=customer_checkin_management")
        logging.info("Person linked to both daily and customer groups.")
else:
    logging.info("Person found in daily group.")

logging.info("Process completed successfully.")